In [2]:
%load_ext autoreload

In [1]:
!python -V

Python 3.12.9


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [21]:
BASE_URL = "../"

In [23]:
import mlflow


mlflow.set_tracking_uri(f"sqlite:///{BASE_URL}/mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/10/20 17:50:42 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/20 17:50:42 INFO mlflow.store.db.utils: Updating database tables
2025-10-20 17:50:42 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-10-20 17:50:42 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-10-20 17:50:42 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-10-20 17:50:42 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/10/20 17:50:42 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/crincon/Documents/study/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1761000642488, experiment_id='1', last_update_time=1761000642488, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [24]:
TRAIN_DATA='https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-01.parquet'
TEST_DATA='https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-02.parquet'

In [25]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [26]:
df_train = read_dataframe(TRAIN_DATA)
df_val = read_dataframe(TEST_DATA)

In [27]:
len(df_train), len(df_val)

(46307, 44218)

In [28]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [29]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [30]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

37.192880963937924

In [32]:
with open(f'{BASE_URL}/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [33]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", TRAIN_DATA)
    mlflow.log_param("valid-data-path", TEST_DATA)

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path=f"{BASE_URL}/models/lin_reg.bin", artifact_path="models_pickle")

In [34]:
import xgboost as xgb

In [35]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [36]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [37]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [38]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.13576                              
[1]	validation-rmse:7.40557                              
  0%|             | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [17:57:08] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:6.86661                              
[3]	validation-rmse:6.47448                              
[4]	validation-rmse:6.19171                              
[5]	validation-rmse:5.98688                              
[6]	validation-rmse:5.84310                              
[7]	validation-rmse:5.74150                              
[8]	validation-rmse:5.66891                              
[9]	validation-rmse:5.61795                              
[10]	validation-rmse:5.58180                             
[11]	validation-rmse:5.55447                             
[12]	validation-rmse:5.53507                             
[13]	validation-rmse:5.51990                             
[14]	validation-rmse:5.50957                             
[15]	validation-rmse:5.49946                             
[16]	validation-rmse:5.49090                             
[17]	validation-rmse:5.48496                             
[18]	validation-rmse:5.48033                             
[19]	validatio

/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [17:57:19] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.01959                              
[1]	validation-rmse:6.06672                              
[2]	validation-rmse:5.66331                              
[3]	validation-rmse:5.49379                              
[4]	validation-rmse:5.42427                              
[5]	validation-rmse:5.39809                              
[6]	validation-rmse:5.38435                              
[7]	validation-rmse:5.37071                              
[8]	validation-rmse:5.36758                              
[9]	validation-rmse:5.36518                              
[10]	validation-rmse:5.36544                             
[11]	validation-rmse:5.36767                             
[12]	validation-rmse:5.36620                             
[13]	validation-rmse:5.36467                             
[14]	validation-rmse:5.36511                             
[15]	validation-rmse:5.36639                             
[16]	validation-rmse:5.36557                             
[17]	validatio

...

[5]	validation-rmse:6.38470                              
[6]	validation-rmse:6.19244                              
[7]	validation-rmse:6.04101                              
[8]	validation-rmse:5.92412                              
[9]	validation-rmse:5.83111                              
[10]	validation-rmse:5.75993                             
[11]	validation-rmse:5.70315                             
[12]	validation-rmse:5.65803                             
[13]	validation-rmse:5.62365                             
[14]	validation-rmse:5.59692                             
[15]	validation-rmse:5.57527                             
[16]	validation-rmse:5.55750                             
[17]	validation-rmse:5.54336                             
[18]	validation-rmse:5.53045                             
[19]	validation-rmse:5.52093                             
[20]	validation-rmse:5.51289                             
[21]	validation-rmse:5.50593                             
[22]	validatio

/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [18:04:00] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:8.35865                              
[2]	validation-rmse:8.03744                              
[3]	validation-rmse:7.74839                              
[4]	validation-rmse:7.48832                              
[5]	validation-rmse:7.25484                              
[6]	validation-rmse:7.04611                              
[7]	validation-rmse:6.85923                              
[8]	validation-rmse:6.69270                              
[9]	validation-rmse:6.54426                              
[10]	validation-rmse:6.41204                             
[11]	validation-rmse:6.29508                             
[12]	validation-rmse:6.19129                             
[13]	validation-rmse:6.09905                             
[14]	validation-rmse:6.01806                             
[15]	validation-rmse:5.94695                             
[16]	validation-rmse:5.88354                             
[17]	validation-rmse:5.82820                             
[18]	validatio

/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [18:04:18] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:8.18041                              
[2]	validation-rmse:7.79990                              
[3]	validation-rmse:7.46562                              
[4]	validation-rmse:7.17409                              
[5]	validation-rmse:6.92038                              
[6]	validation-rmse:6.70052                              
[7]	validation-rmse:6.51134                              
[8]	validation-rmse:6.34741                              
[9]	validation-rmse:6.20670                              
[10]	validation-rmse:6.08552                             
[11]	validation-rmse:5.98252                             
[12]	validation-rmse:5.89463                             
[13]	validation-rmse:5.81954                             
[14]	validation-rmse:5.75538                             
[15]	validation-rmse:5.69997                             
[16]	validation-rmse:5.65345                             
[17]	validation-rmse:5.61404                             
[18]	validatio

In [39]:
mlflow.xgboost.autolog(disable=True)

In [46]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open(f"{BASE_URL}/models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(f"{BASE_URL}/models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:8.56745
[1]	validation-rmse:8.10033


/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [18:34:42] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059
[10]	validation-rmse:6.00193
[11]	validation-rmse:5.90700
[12]	validation-rmse:5.82752
[13]	validation-rmse:5.76123
[14]	validation-rmse:5.70306
[15]	validation-rmse:5.65560
[16]	validation-rmse:5.61619
[17]	validation-rmse:5.58065
[18]	validation-rmse:5.55157
[19]	validation-rmse:5.52734
[20]	validation-rmse:5.50858
[21]	validation-rmse:5.48872
[22]	validation-rmse:5.47379
[23]	validation-rmse:5.45957
[24]	validation-rmse:5.44705
[25]	validation-rmse:5.43706
[26]	validation-rmse:5.42713
[27]	validation-rmse:5.41891
[28]	validation-rmse:5.41283
[29]	validation-rmse:5.40819
[30]	validation-rmse:5.40244
[31]	validation-rmse:5.39854
[32]	validation-rmse:5.39446
[33]	validation-rmse:5.39004
[34]	validation-rmse:5.38685
[35]	validation-rmse:5.38264
[36]	validation-rmse:5

2025/10/20 18:34:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [18:34:52] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1570: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)


ModuleNotFoundError: No module named '_lzma'

In [43]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", TRAIN_DATA)
        mlflow.log_param("valid-data-path", TEST_DATA)
        mlflow.log_artifact(f"{BASE_URL}/models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

2025/10/20 18:25:37 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named '_lzma'
2025/10/20 18:25:41 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named '_lzma'
2025/10/20 18:28:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named '_lzma'
/Users/crincon/Documents/study/mlops-zoomcamp/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/20 18:28:42 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: No module named '_lzma'
